In [1]:
import pandas as pd
import numpy as np
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords 
import operator
import numpy 

In [2]:
train_data = pd.read_csv('train.dat', sep='\t',header=None, names = ["label","extract"])
test_data = pd.read_csv('test.dat', sep='\t',header=None, names = ["extract"])

In [3]:
train_data.head(10) 

,label,extract
0,4,Catheterization laboratory events and hospital...
1,5,Renal abscess in children. Three cases of rena...
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...
3,5,Subclavian artery to innominate vein fistula a...
4,4,Effect of local inhibition of gamma-aminobutyr...
5,1,Infection during chronic epidural catheterizat...
6,5,Mediastinal tracheostomy using a pectoralis ma...
7,5,Tumefactive fibroinflammatory lesion of the ex...
8,3,Multiple representations contribute to body kn...
9,5,Increasing asthma prevalence in a rural New Ze...


In [4]:
test_data.head(10) 

,extract
0,Excision of limbal dermoids. We reviewed the c...
1,Bell's palsy. A diagnosis of exclusion. In cas...
2,Retained endobronchial foreign body removal fa...
3,Recurrent buccal space abscesses: a complicati...
4,Intracranial fibromatosis. Fibromatoses are un...
5,The effect of intrathecal morphine on somatose...
6,The 29th Rovenstine lecture: clinical challeng...
7,Mortality in patients treated with flecainide ...
8,Approaches to immunotherapy of cancer: charact...
9,Retinal artery obstruction and atheromas assoc...


In [5]:
train_data.describe() 


,label
count,14438.000000
mean,3.335296
std,1.552432
min,1.000000
25%,2.000000
50%,4.000000
75%,5.000000
max,5.000000


In [6]:
test_data.describe()

,extract
count,14442
unique,11279
top,Long-term results after lateral cranial base s...
freq,4


In [7]:
extracted_data = train_data["extract"]
train_docs = [l.split() for l in extracted_data]
stop_words = set(stopwords.words("english"))
minlen =  4

def preprocess_doc(docs):
    for ind1,doc in enumerate(docs):
        for ind2,word in enumerate(doc):
            if (len(word) >= minlen) and (word not in stop_words) and word.isalnum():
                docs[ind1][ind2] = word.lower()
#                 ls = LancasterStemmer()
#                 docs[ind1][ind2] = ls.stem(word)
                lm = WordNetLemmatizer()
                docs[ind1][ind2] = lm.lemmatize(docs[ind1][ind2])
            else:
                del docs[ind1][ind2]
    return docs



# def filter_Len(docs, minlen):
#     return [ [t.lower() for t in d if len(t) >= minlen] for d in docs ]

# def stop_filter(docs):
#     return [ [t for t in d if t not in stop_words ] for d in docs ]

# def stem_words(docs):
#     for ind1,doc in enumerate(docs):
#         for ind2,word in enumerate(doc):
#             ls = LancasterStemmer()
#             docs[ind1][ind2] = ls.stem(word)
#     return docs

# def lemmatize_words(docs):
#     for ind1,doc in enumerate(docs):
#         for ind2,word in enumerate(doc):
#             lm = WordNetLemmatizer()
#             docs[ind1][ind2] = lm.lemmatize(word)
#     return docs



In [8]:
# #alpha_num_docs = remove_special_characters(filtered_docs)
# stop_filtered_docs = stop_filter(docs)
# filtered_docs = filter_Len(stop_filtered_docs,4)
# stemed_docs = stem_words(filtered_docs)
# lemm_docs = lemmatize_words(stemed_docs)

In [9]:
# print(len(docs[0]), docs[0][:20])
# print(len(filtered_docs[0]), filtered_docs[0][:20])
# print(len(stop_filtered_docs[0]), stop_filtered_docs[0][:20])
# print(len(stemed_docs[0]), stemed_docs[0][:20])
# print(len(lemm_docs[0]), lemm_docs[0][:20])

In [10]:
preprocessed_doc =preprocess_doc(train_docs)

In [11]:
print(len(train_docs[0]), train_docs[0][:20])
print(len(preprocessed_doc[0]), preprocessed_doc[0][:20])


155 ['catheterization', 'laboratory', 'event', 'hospital', 'outcome', 'direct', 'angioplasty', 'acute', 'myocardial', 'infarction', 'assess', 'safety', 'direct', 'infarct', 'angioplasty', 'without', 'antecedent', 'thrombolytic', 'catheterization', 'laboratory']
155 ['catheterization', 'laboratory', 'event', 'hospital', 'outcome', 'direct', 'angioplasty', 'acute', 'myocardial', 'infarction', 'assess', 'safety', 'direct', 'infarct', 'angioplasty', 'without', 'antecedent', 'thrombolytic', 'catheterization', 'laboratory']


In [12]:
extracted_test_data = test_data["extract"]
test_docs = [l.split() for l in extracted_test_data]


# stop_filtered_test_docs = stop_filter(test_docs)
# filtered_test_docs = filter_Len(stop_filtered_test_docs,3)
# stemed_test_docs = stem_words(filtered_test_docs)
# lemm__test_docs = lemmatize_words(stemed_test_docs)


In [13]:
preprocessed_test_doc =preprocess_doc(test_docs)

In [14]:
print(len(test_docs[0]), test_docs[0][:20])
print(len(preprocessed_test_doc[0]), preprocessed_test_doc[0][:20])


78 ['excision', 'limbal', 'We', 'reviewed', 'clinical', 'file', '10', 'patient', 'had', 'undergone', 'excision', 'unilateral', 'epibulbar', 'limbal', 'Preoperatively,', 'of', 'affected', 'eye', 'worse', 'visual']
78 ['excision', 'limbal', 'We', 'reviewed', 'clinical', 'file', '10', 'patient', 'had', 'undergone', 'excision', 'unilateral', 'epibulbar', 'limbal', 'Preoperatively,', 'of', 'affected', 'eye', 'worse', 'visual']


In [15]:
cls = train_data["label"]
cls[0:10]

0    4
1    5
2    2
3    5
4    4
5    1
6    5
7    5
8    3
9    5
Name: label, dtype: int64

In [16]:
type(extracted_data)

pandas.core.series.Series

In [17]:

from collections import Counter
from scipy.sparse import csr_matrix
def build_matrix(docs):
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat, idx

def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )



In [18]:
#mat, idx = build_matrix(train_docs)
train_mat, idx = build_matrix(preprocessed_doc)
#csr_info(mat)
csr_info(train_mat)

 [nrows 14438, ncols 70735, nnz 1161420]


In [19]:
def build_matrix_test(docs,idx):
    nrows = len(docs)
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            temp = idx.get(k,-1)
            if temp != -1:
                ind[j+n] = temp
                val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

In [20]:
#test_mat = build_matrix_test(test_docs,idx)
test_mat = build_matrix_test(preprocessed_test_doc,idx)
#csr_info(test_mat)
csr_info(test_mat)

 [nrows 14442, ncols 70735, nnz 1182300]


In [21]:
from collections import defaultdict
def csr_idf(mat, copy=False, **kargs):
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

def csr_l2normalize(mat, copy=False, **kargs):

    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat


In [23]:
train_mat1 = csr_idf(train_mat, copy=True)
train_mat2= csr_l2normalize(train_mat1, copy=True)
print("mat1:", train_mat[12,:20].todense(), "\n")
print("mat1:", train_mat.shape, "\n")

print("mat1:", train_mat1[12,:20].todense(), "\n")
print("mat1:", train_mat1.shape, "\n")

print("mat1:", train_mat2[12,:20].todense(), "\n")
print("mat1:", train_mat2.shape, "\n")


mat1: [[0. 0. 0. 0. 0. 0. 0. 5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5.]] 

mat1: (14438, 70735) 

mat1: [[ 0.          0.          0.          0.          0.          0.
   0.         10.86061618  0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          3.00931134]] 

mat1: (14438, 70735) 

mat1: [[0.         0.         0.         0.         0.         0.
  0.         0.15118496 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.04189105]] 

mat1: (14438, 70735) 



In [24]:
test_mat1 = csr_idf(test_mat, copy=True)
test_mat2 = csr_l2normalize(test_mat1, copy=True)
print("mat1:", test_mat[15,:20].todense(), "\n")
print("mat2:", test_mat1[15,:20].todense(), "\n")
print("mat3:", test_mat2[15,:20].todense())


mat1: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 2.]] 

mat2: [[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  2.07460628 1.20023662]] 

mat3: [[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.0380577  0.02201779]]


In [25]:
def cosine_similarity(matrix_a,matrix_b):
    cosine_sim = matrix_a*(matrix_b.T)
    return cosine_sim


In [26]:
def classlabel_voting(cos_sim_list, k, epsilon):
        k_neighbours = cos_sim_list[:k]
        class_label_dict = {}
        for neighbour in k_neighbours:
            classlabel = neighbour[0]
            sim_measure = neighbour[1]
            if sim_measure < epsilon:
                continue
            if classlabel in class_label_dict:
                class_label_dict[classlabel] += 1
            else :
                class_label_dict[classlabel] = 1
        sorted_class_labels_list = sorted(class_label_dict.items(), key=operator.itemgetter(1), reverse=True)
        if len(sorted_class_labels_list) > 0:
            req_class_label = sorted_class_labels_list[0][0]
        else:
            req_class_label = k_neighbours[0][0]
        return req_class_label

In [27]:
def knn_algorithm(train_matrix, test_matrix, class_labels, k, epsilon):
        nrows = test_matrix.shape[0]
        predictions_file = open("smsn.dat","w")
        for i in range(nrows):
            similarity_matrix = cosine_similarity(train_matrix, test_matrix[i])
#             print(similarity_matrix)
            sim_map = {}
            nrows_sim_matrix = similarity_matrix.shape[0]
            for j in range(nrows_sim_matrix):
                sim_map[class_labels[j]] = similarity_matrix[j]             
            sorted_sim_list = sorted(sim_map.items(), key=operator.itemgetter(1), reverse=True)
            res_class_label = classlabel_voting(sorted_sim_list, k, epsilon)
            predictions_file.write(str(res_class_label))
            predictions_file.write("\n")
        predictions_file.close()

In [28]:
k = 12
epsilon = 0.7

In [ ]:
knn_algorithm(train_mat2, test_mat2, cls, k, epsilon)


C:\Users\Rutul Thakkar\Anaconda3\lib\site-packages\scipy\sparse\compressed.py:274: SparseEfficiencyWarning: Comparing a sparse matrix with a scalar greater than zero using < is inefficient, try using >= instead.
  warn(bad_scalar_msg, SparseEfficiencyWarning)


In [ ]:
# def cosine_similarity(testmat,trainmat):       
#         cosine_sim_mat = trainmat.dot(testmat.T)
#         return cosine_sim_mat

# knn = 3
# epsilon =0.1
# #%%
# def knn_algorithm(train_matrix, test_matrix, class_labels_arr, knn, epsilon):
#         pred_format_file = open("myfinaltestprg1pred.dat","w") 
#         cos_sim_matrix = cosine_similarity(test_matrix, train_matrix)    
#         ##rows of sim matrix is same as number of rows of trainingset and cols are same as number of rows of test set
#         ncols_sim_mat = cos_sim_matrix.shape[1] # this is same as number of rows in test matrix   
#         for i in range(ncols_sim_mat):
#             #get first column values
#             row_sim_array = cos_sim_matrix[:,i].toarray()
#             npsimarr = numpy.array(row_sim_array.copy())          
#             dict1_labels = {}
#             for i in range(knn):
#                 maxval = numpy.amax(npsimarr, axis=0)
#                 #print(maxval[0],end=",")
#                 if maxval[0] < epsilon:
#                     continue
#                 indexarr = numpy.where(npsimarr == numpy.amax(npsimarr))
#                 index = indexarr[0][0]
#                 #print(index,end=",")
#                 npsimarr[index][0] = 0.0
#                 classlabel = class_labels_arr[index]
#                 if classlabel in dict1_labels:
#                     dict1_labels[classlabel] += 1
#                 else:
#                     dict1_labels[classlabel] = 1
#             #print(dict1_labels) 
#             sorted_labels = sorted(dict1_labels.items(), key=operator.itemgetter(1), reverse=True)
#             #print(sorted_labels)
#             finallabel = sorted_labels[0][0]
#             print(finallabel)
#             pred_format_file.write(finallabel)           
#             pred_format_file.write("\n")            
#         pred_format_file.close()